In [2]:
"""Import from parent directory."""
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path: sys.path.append(module_path)

In [6]:
'''
We'll start off by showing how our client extracts the relevant information from
a data provider.
'''
from agora_dp import post_dataset
post_dataset("path/to/iot/data", "iot")

"\nWe'll start off by showing how our client extracts the relevant information from\na data provider.\n"

In [1]:
'''
Now we'll see how a model developer would view the information.
'''
import agora_mt as amt
amt.get_dataset("iot")

'test123'

In [ ]:
'''
For this proof of concept, our data providers are running on AWS.
But running a data provider is very easy, as it's a simple Unix service.
Let's peek inside the classes which make up the Unix service to see how it behaves in more detail.
'''
from core.listener import DMLListener
config = {
  "dataset_path": "datasets/power",
  "scheduler_config": {
    "frequency_in_mins": 1,
    "num_processes": 4
  },
  "runner_config": {
    "weights_directory": "weights"
  }
}

listener = DMLListener(config)

In [ ]:
'''
Let's make an appropriate model for this data.
'''
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
from models.generic_model import GenericKerasModel

class KerasLSTM(GenericKerasModel):
    def __init__(self, is_training=False):
        self.is_training = is_training
        self.model = self.build_model()
        if is_training:
            self.compile_model()

    def build_model(self):
        model = Sequential()
        model.add(LSTM(160, input_shape = (1, 686), return_sequences=True))
        model.add(LSTM(200))
        model.add((Dense(1)))
        model.summary()
        return model

    def compile_model(self):
        self.model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mape'])

In [ ]:
from core.developer import DMLDeveloper

model = KerasLSTM(is_training=True).model
developer = DMLDeveloper()
# developer.checkBalance()
developer.deploy_with_model(model)

In [ ]:
'''
That's it! Very simply, we've trained a model over decentralized datasets.
We haven't specified any stopping conditions, so this model will only train for 1 cycle.
'''